In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from datetime import timedelta
import seaborn as sns
%matplotlib inline
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE
import sklearn.metrics
from datetime import datetime

# Analysis of Cycling Data

## Loading Data

The first dataset is an export of my ride data from [Strava](https://strava.com/), an online social 
network site for cycling and other sports.  This data is a log of every ride since the start of 2018
and contains summary data like the distance and average speed.  It was exported using 
the script `stravaget.py` which uses the stravalib module to read data. Some details of
the fields exported by that script can be seen in [the documentation for stravalib](https://pythonhosted.org/stravalib/api.html#stravalib.model.Activity). 

The exported data is a CSV file so that's easy to read, however the date information in the file is 
recorded in a different timezone (UTC) so we need to do a bit of conversion.  In reading the data I'm
setting the index of the data frame to be the datetime of the ride. 

In [ ]:
strava = pd.read_csv('data/strava_export.csv', index_col='date', parse_dates=True)
strava.index = strava.index.tz_convert('Australia/Sydney')

The second dataset comes from an application called [GoldenCheetah](https://www.goldencheetah.org/) which provides
some analytics services over ride data.  This has some of the same fields but adds a lot of analysis of the 
power, speed and heart rate data in each ride.  This data overlaps with the Strava data but doesn't include all 
of the same rides. 

Again we create an index using the datetime for each ride, this time combining two columns in the data (date and time) 
and localising to Sydney so that the times match those for the Strava data. 

In [ ]:
cheetah = pd.read_csv('data/cheetah.csv', skipinitialspace=True)
cheetah.index = pd.to_datetime(cheetah['date'] + ' ' + cheetah['time'])
cheetah.index = cheetah.index.tz_localize('Australia/Sydney')

The GoldenCheetah data contains many many variables (columns) and I won't go into all of them here. Some
that are of particular interest for the analysis below are:

Here are definitions of some of the more important fields in the data. Capitalised fields come from the GoldenCheetah data
while lowercase_fields come from Strava. There are many cases where fields are duplicated and in this case the values
should be the same, although there is room for variation as the algorithm used to calculate them could be different
in each case. 

  * Duration - overall duration of the ride, should be same as elapsed_time
  * Time Moving - time spent moving (not resting or waiting at lights), should be the same as moving_time
  * Elevation Gain - metres climbed over the ride
  * Average Speed - over the ride
  * Average Power - average power in watts as measured by a power meter, relates to how much effort is being put in to the ride, should be the same as  * average_watts' from Strava
  * Nonzero Average Power - same as Average Power but excludes times when power is zero from the average
  * Average Heart Rate - should be the same as average_heartrate
  * Average Cadence - cadence is the rotations per minute of the pedals
  * Average Temp - temperature in the environment as measured by the bike computer (should be same as average_temp)
  * VAM - average ascent speed - speed up hills
  * Calories (HR) - Calorie expendature as estimated from heart rate data
  * 1 sec Peak Power - this and other  'Peak Power' measures give the maximum power output in the ride over this time period.  Will be higher for shorter periods. High values in short periods would come from a very 'punchy' ride with sprints for example.
  * 1 min Peak Hr - a similar measure relating to Heart Rate
  * NP - Normalised Power, a smoothed average power measurement, generally higher than Average Power 
  * TSS - Training Stress Score, a measure of how hard a ride this was
  * device_watts - True if the power (watts) measures were from a power meter, False if they were estimated
  * distance - distance travelled in Km
  * kudos - likes from other Strava users (social network)
  * workout_type - one of  'Race',  'Workout' or  'Ride'
  
  
Some of the GoldenCheetah parameters are defined [in thier documentation](https://github.com/GoldenCheetah/GoldenCheetah/wiki/UG_Glossary).  

## Your Tasks

Your first task is to combine these two data frames using the [`join` method of Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#joining-on-index).   The goal is to keep only those rows of data 
that appear in __both__ data frames so that we have complete data for every row.  

In [ ]:
ride_data = strava.join(cheetah)

In [ ]:
allwork = ride_data[ride_data.workout_type == 'Workout']

In [ ]:
ride_data.dropna(subset = ["device_watts"], inplace = True)

In [ ]:
rd = ride_data.dropna()

In [ ]:
rd = rd[rd.device_watts]

In [ ]:
rd.drop(['filename', 'date'], axis = 1, inplace = True)

In [ ]:
z=rd[['Distance', 'Time Moving' , 'Average Speed', 'Average Heart Rate', 'average_watts','NP', 'TSS', 'Elevation Gain' ]]
##scattermat = pd.plotting.scatter_matrix(z, marker = ".", diagonal='kde', figsize=(15,15), grid = True)

In [ ]:
rd.drop(['elapsed_time', 'moving_time'], axis = 1, inplace=True)

In [ ]:
mins = rd.Duration/60
rd['Duration_minutes'] = mins
rd.head()

In [ ]:
sns.set_style('whitegrid')
sns.set(font_scale = 1.5)
seapair = sns.pairplot(z, diag_kind='kde', palette= 'bright', height= 4)

## Required Analysis

1. Remove rides with no measured power (where device_watts is False) - these are commutes or MTB rides
* Look at the distributions of some key variables: time, distance, average speed, average power, TSS. Are they normally distributed? Skewed? 
* Explore the relationships between the following variables. Are any of them corrolated with each other (do they vary together in a predictable way)? Can you explain any relationships you observe?  
    * Distance
    * Moving Time
    * Average Speed
    * Heart Rate
    * Power (watts)
    * Normalised power (NP)
    * Training Stress Score
    * Elevation Gain
* We want to explore the differences between the three categories: `Race`, `Workout` and `Ride`.
    * Use scatter plots with different colours for each category to explore how these categories differ.  
    * Use histograms or box plots to visualise the different distributions of a variable for the three categories.
    * In both cases, experiment with different variables but only include those that are interesting in your final notebook (if none are interesting, show us a representative example).


## Challenge

* What leads to more `kudos`? Is there anything to indicate which rides are more popular? Explore the relationship between the main variables and kudos. Show a plot and comment on any relationship you observe. 

* Generate a plot that summarises the number of km ridden each month over the period of the data. Overlay this with the _sum_ of the Training Stress Score and the _average_ of the Average Speed to generate an overall summary of activity.

* Generate a similar graph but one that shows the activity over a given month, with the sum of the values for each day of the month shown.  So, if there are two rides on a given day, the graph should show the sum of the distances etc for these rides.

Hint: to generate these summary plots you need to use the [timeseries/date functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html) in Pandas to generate a new data frame containing the required data.  

__Note:__ once you have completed these steps you can remove this cell.  Use the text as a starting point for the documentation of your workflow and discussion of results.


In [ ]:
race = rd[rd.workout_type == "Race"]
ride = rd[rd.workout_type == "Ride"]
work= rd[rd.workout_type == "Workout"]
x = ['Distance', 'kudos', 'Time Moving', 'Average Heart Rate', 'Duration_minutes', 'Average Power', 'Work', 'Average Speed', 'Calories (HR)']

In [ ]:
race = race[x]
ride = ride[x]
work = work[x]

# Analysis of Data #

## Scatterplots for Comparison of variables in Race, Ride and Workout ##

* ### Race ###

In [ ]:
plt.figure(figsize = (25,10))
sns.set_style('darkgrid')
with sns.xkcd_palette(['green', 'purple', 'red']) :
    sns.lmplot(x = 'Distance', y = 'Duration_minutes', data = rd, fit_reg= False,
           hue = 'workout_type', legend_out= True, scatter_kws= {"s" : 20}, height = 7)

There is evidence of a **linear** relationship between the variables Distance and Duration for the category Race and Ride, however, we do not have enough data for Workouts to hypothesize a relationship between Duration and Distance. 

In [ ]:
plt.figure(figsize = (25,10))
sns.set_style('darkgrid')
with sns.xkcd_palette(['green', 'purple', 'red']) :
    sns.lmplot(x = 'Distance', y = 'Duration_minutes', data = rd, fit_reg= True,
           hue = 'workout_type', legend_out= True, scatter_kws= {"s" : 20}, height = 7)
    plt.ylabel('Duration (in minutes)')

It is to be noted that the variable **x** is **Distance**, while variable **y** is **Duration**. It can be explained by the regression lines that as distance increases, so does duration. It is to be noted that the slope is highest for the category *Ride*, meaning that, the average speed during a Ride should be less compared to the other two variables. It can be assumed from the regression line that the average speed in a Race is higher than a Ride. It is also seen that Workout has the least slope, meaning, in theory it should have the highest average speed. But one should keep in mind that the Workout category only has 5 observations, and the distance and duration is much lesser than the other two categories, which is a contributing factor in a result that may be skewed

In [ ]:
boxdist = rd.boxplot(by = 'workout_type', column = ['distance'], figsize = (7,7))

The category **ride** has the highest median distance covered per ride. It also has the highest range, meaning that it had a significant number of short and long distance rides. The median distace for **races** and **workouts** are almost the same, however, the spread for **races** is larger. They also have some outliers 

In [ ]:
avgpowerbox = rd.boxplot(by = 'workout_type', column = ['Average Power'], figsize = (7,7))

Median power is the highest for **races**. The median for **workouts** is quite close to the maximum value, indicating that the power output was mostly high. However, it can be traced to the fact that the number of observations is quite low. **Rides** have the highest spread of power output, but have the lowest median

In [ ]:
avgspeedbox = rd.boxplot(by = 'workout_type', column = ['Average Speed'], figsize = (7,7))

The distribution of speed is compact for all three categories. **Workouts** have the lowest spread, but have an outlier that is significantly less than its lower quartile. **Races** have the highest median value, whereas **rides** have the lowest median and highest spread

In [ ]:
dur_hist = plt.hist(rd['Duration'], bins = 20)
z = plt.title("Distribution of Duration as a Histogram")

In [ ]:
dist_hist = plt.hist(rd['Distance'], bins = 20)
z = plt.title("Distance distribution as a histogram")

The distribution of distance seems to be **bimodal**, however, the second mode is much more distinct compared to the first mode. The observations towards the right end of the histogram are very small, meaning that high distance rides were less frequent.

In [ ]:
avg_speed_hist = plt.hist(rd['Average Speed'], bins = 20)
z = plt.title("Distribution of Average Speed")

The distribution of average speed is **skewed to the right**. This means that the **mean is higher than the median** of the data. The number of high average speed rides were significantly less, which resulted in a high mean, but did not affect the median value much

In [ ]:
avg_power_hist = plt.hist(rd['Average Power'], bins = 20)
z = plt.title("Distribution of Average Power")

The distribution for average power is **normally distributed**

In [ ]:
tss_hist = plt.hist(rd['TSS'], bins = 20)
z = plt.title("Distribution of TSS")

The distribution for training stress score is **bimodal**. There are some high scored observations, but their frequency is quite low

In [ ]:
rd['Month'] = rd.index.month

In [ ]:
months = {1:'January', 2 : 'February', 3 : 'March', 4: 'April', 5:'May', 6: 'June', 7: 'July', 8 : 'August', 9 : 'September', 10 : 'October', 11: 'November', 12: 'December'}
rd['Month'] = rd['Month'].map(months)

In [ ]:
plt.figure(figsize = (20,12))
sns.barplot(y = 'Distance', x = 'Month', data = rd, ci = None, orient='v', palette= 'gist_heat')

This graph shows the total distance ridden during each month. It can be noted that in **October**, over 60km was covered in bike rides. **June** and **August** had the least distance covered, with both measuring a little over 30km

In [ ]:
kudos = rd['kudos']
feature_cols = np.array(['Distance', 'Time Moving', 'Average Heart Rate', 'Duration_minutes', 'Average Power', 'Work', 'Average Speed', 'Calories (HR)', 'Elevation Gain'])
X = rd[feature_cols]

In [ ]:
estimator = LinearRegression()
selector  = RFE(estimator, 9)
selector = selector.fit(X, kudos)

In [ ]:
pred = selector.predict(X)

In [ ]:
sklearn.metrics.r2_score(pred, kudos)

In [ ]:
sns.distplot(kudos, hist = False, kde_kws={'color' : 'r'})
sns.distplot(pred, hist = False, kde_kws = {'color' : 'k'})

An $ R^2 $ score of $ 0.6 $ or $ 60 \%\ $ indicates that the predictor model is a decent fit of our actual model, although it does deviate from it at certain instances.

The **red** graph depicts the distribution of kudos from the actual data. The **black** graph depicts the prediction of kudos with the algorithm. It can be seen that the predictor follows the data closely upto most extent, except for a spike around **20** units, where our predictor's estimates are significantly higher than the actual model

In [ ]:
supp = selector.get_support()

In [ ]:
feature_cols[supp]

In [ ]:
print("Regression Coefficients \n")
for i in range(len(feature_cols[supp])):
    print(feature_cols[supp][i] , ":", round(selector.estimator_.coef_[i], 5))

**Distance** has a regression coefficient of 0.22, which implies that the change in value of distance has the most impact (out of these given variables), on the value of kudos.

In [ ]:
months = []
for i in rd.index:
    months.append(str(i.month) + "/" + str(i.year))

In [ ]:
months = np.array(months)

In [ ]:
dat = rd[['TSS', 'Average Speed']]

In [ ]:
dat = dat.reset_index()

In [ ]:
dat['months'] = months

In [ ]:
dat.drop('date', axis = 1, inplace = True)

In [ ]:
tss_sum = dict()
for x in range(len(dat.months)):
    if (dat.months[x]) in tss_sum:
        tss_sum[dat.months[x]] += dat.TSS[x]
    else:
        tss_sum[dat.months[x]] = dat.TSS[x]s

In [ ]:
l1 = []
for i in tss_sum:
    for j in dat.months:
        if (i == j):
            l1.append(tss_sum[j])
list1 = []
for j in l1:
    x = j/100
    list1.append(x)

In [ ]:
avg_spd = dict()
for x in range(len(dat.months)):
    if (dat.months[x]) in avg_spd:
        avg_spd[dat.months[x]] += dat['Average Speed'][x]
    else:
        avg_spd[dat.months[x]] = dat['Average Speed'][x]

In [ ]:
counter = dict()
for i in dat.months:
    if i in counter:
        counter[i] += 1
    else:
        counter[i] = 1

In [ ]:
avg_avg_speed = []
for i in avg_spd:
    avg_avg_speed.append(avg_spd[i]/counter[i])

In [ ]:
l2 = []
for i in avg_spd:
    for j in dat.months:
        if (i == j):
            l2.append(avg_spd[j])

In [ ]:
list2 = []
for i in l2:
    j = i/100
    list2.append(j)

In [ ]:
dat['Monthly TSS Sum (in 100\'s)'] = list1

In [ ]:
dat['Monthly Average Avg Speed (in 100 km/h)'] = list2

In [ ]:
x = np.array(rd.Distance)
dat['Distance'] = x

In [ ]:
plt.figure(figsize = (18,7))
plt.plot(dat.index,dat['Distance'], c = 'r')
plt.plot(dat.index, dat['Monthly TSS Sum (in 100\'s)'], c = 'g')
plt.plot(dat.index, dat['Monthly Average Avg Speed (in 100 km/h)'])

In [ ]:
kudos = pd.DataFrame(kudos)

In [ ]:
weekday = kudos.index.weekday

In [ ]:
weekstatus = []
for i in weekday:
    if (i > 5):
        weekstatus.append(0)
    else:
        weekstatus.append(1)
kudos['isWeekend'] = weekstatus
kudos['isWeekend'] = kudos['isWeekend'].astype('int64')

In [ ]:
logreg = LogisticRegression()
feature_col = ['kudos']
X = kudos[feature_col]
y = kudos['isWeekend']
logreg.fit(X,y)

In [ ]:
probs = logreg.predict_proba(X)[:,1]

In [ ]:
plt.scatter(X,probs)

In [ ]:
feat_cols = np.append(feature_cols,'kudos')

In [ ]:
feat_graph = rd[feat_cols]

In [ ]:
feat_graph = feat_graph.corr()

In [ ]:
plt.figure(figsize = (5,5))
plt.style.use('default')
heatmap = sns.heatmap(feat_graph, square = True, cmap="YlGnBu")

The heatmap shows correlation of some of the key variables in the dataset. The darker shades imply a higher correlation value, as compared to lighter shades. Here, we are not concerned with the diagonal running from the top-left to the bottom right, as it just shows the correlation of each variable with itself (which is always 1). 

It is important to note that correlation does not imply causation. If two variables are correlated, it does not mean that one causes the other.